In [5]:
text = 'Kupac: Koliko kosta Bosch GBH 141? Prodavac: Postovani, Bosch GBH 141 kosta 10000 RSD.'
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

stoi = { ch:i for i,ch in enumerate(chars) } # dict comperhension eg. A:0
itos = { i:ch for i,ch in enumerate(chars) } # dict comperhension eg. 0:A

def encode(chars):
    return [stoi[c] for c in chars]

def decode(chars):
    return ''.join([itos[i] for i in chars])

encoded = encode('Prodavac: Postovani')
decoded = decode(encoded)
print(encoded)
print(decoded)

 ,.014:?BDGHKPRSacdhiklnoprstuv
31
[13, 26, 24, 18, 16, 30, 16, 17, 6, 0, 13, 24, 27, 28, 24, 30, 16, 23, 20]
Prodavac: Postovani


In [10]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data)

torch.Size([86]) torch.int64
tensor([12, 29, 25, 16, 17,  6,  0, 12, 24, 22, 20, 21, 24,  0, 21, 24, 27, 28,
        16,  0,  8, 24, 27, 17, 19,  0, 10,  8, 11,  0,  4,  5,  4,  7,  0, 13,
        26, 24, 18, 16, 30, 16, 17,  6,  0, 13, 24, 27, 28, 24, 30, 16, 23, 20,
         1,  0,  8, 24, 27, 17, 19,  0, 10,  8, 11,  0,  4,  5,  4,  0, 21, 24,
        27, 28, 16,  0,  4,  3,  3,  3,  3,  0, 14, 15,  9,  2])


In [12]:
n = int(0.9 * (len(data)))
train_data = data[:n]
val_data = data[n:]